In [1]:
import lightkurve as lk
import numpy as np
import pandas as pd
import glob
from astropy.io import ascii
from astropy.table import Table, join, vstack
from astropy.io import fits
import matplotlib.pyplot as plt
from astroquery.mast import Observations, Catalogs
from operator import itemgetter
import glob


# useful functions

In [2]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
 
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos



In [3]:
# unique item finder

def uniquefinder(mylist):
    seen = {}
    dupes = []
    uniq_tics = []
    for x in mylist:
        if x not in seen:
            seen[x] = 1
            uniq_tics.append(x)
        else:
            if seen[x] == 1:
                dupes.append(x)
            seen[x] += 1
    uniques = len(uniq_tics)
    print('there are {} unique tics. Use the first output dictionary'.format(uniques), \
          'to see number of occurances for duplicates')
    return seen, uniq_tics

In [4]:
#useful fcn - compares lists & returns what didnt match
def returnNotMatches(a, b):
    return [[x for x in a if x not in b], [x for x in b if x not in a]] #not in b , not in a

#  Creating table

In [5]:
#Opening stats from each sector 


tic_list = np.load('data/all_dled_tics.npy')
# tic_list = np.load('data/additional_cvz_tics.npy') #list of new tics 
#tic_list = np.load('data/ticadds_needbysector.npy')
# tic_list = np.load('data/good_cvz_tics.npy') #only 10+ sectors



ls1=[]
ls2=[]
ls3=[]
tics=[]
rvars=[]
sector=[]
lsamp1=[]
lsamp2=[]
lsamp3=[]
missing=[]
acfamp1=[]
acfamp2=[]
acfamp3=[]
acf1=[]
acf2=[]
acf3=[]
#tic_list=['349309133',123] #used while testing
for count,tic in enumerate(tic_list):
    sectors = [1,2,3,4,5,6,7,8,9,10,11,12,13]
    for sec in sectors:
        try:
            statsecpath = 'data/SECONDRUN/cleaned_LightCurves/{}/stats.csv'.format(tic)
            stats = pd.read_csv(statsecpath)
        except:
            missing.append(tic)
            continue
        else:
            #sec check b/c additons used float for sec instead on int
            if sec is float:
                sec = int(sec)
            else: 
                sec = sec
            ix = getIndexes(stats,sec)
            #print('ix:',ix,'mysector:',sec, tic,'at idx',count)
            if len(ix) > 0:
                idx = ix[0][0] #gets row index for that sectors row data
                #print('index:',idx)
                lsrp1 = stats.iloc[idx,2]
                ls1amp = stats.iloc[idx,5]
                lsrp2= stats.iloc[idx,3]
                ls2amp = stats.iloc[idx,6]
                lsrp3= stats.iloc[idx,4]
                ls3amp = stats.iloc[idx,7]
                rvar = stats.iloc[idx,1] 
                sect = int(stats.iloc[idx,0])
                acf_rp1 = stats.iloc[idx,8]####
                acf_rp2 = stats.iloc[idx,9]  
                acf_rp3  = stats.iloc[idx,10]
                acf_amp1  = stats.iloc[idx,11]
                acf_amp2 = stats.iloc[idx,12]
                acf_amp3 = stats.iloc[idx,13]
                ls1.append(lsrp1)
                ls2.append(lsrp2)
                ls3.append(lsrp3)
                lsamp1.append(ls1amp)
                lsamp2.append(ls2amp)
                lsamp3.append(ls3amp)
                tics.append(tic)
                rvars.append(rvar)
                sector.append(sect)
                acf1.append(acf_rp1)
                acf2.append(acf_rp2)
                acf3.append(acf_rp3)
                acfamp1.append(acf_amp1)
                acfamp2.append(acf_amp2)
                acfamp3.append(acf_amp3)
            else:
                continue
print('ls:',len(ls1)) # 112219
print(len(tic_list),len(tics)) # 24085 112219
print('lsmap:',len(lsamp1)) # 112219
print(len(missing))#,missing) #182

ls: 112219
24085 112219
lsmap: 112219
182


In [6]:
#creating stats by sectors table
cvzstat_table = Table([tics,sector,rvars,ls1,ls2,ls3,lsamp1,lsamp2,lsamp3,acf1,acf2,acf3,acfamp1,acfamp2,acfamp3],names=('ID','sector','rvar','ls1','ls2','ls3','lsamp1','lsamp2','lsamp3','acf1','acf2','acf3','acfamp1','acfamp2','acfamp3'))
#ascii.write(stat_table_extras,'data/statstable_by_sector_CVZ.dat')
#this is not the final big table, its missing the extras, keep going

In [7]:
cvzstat_table

ID,sector,rvar,ls1,ls2,ls3,lsamp1,lsamp2,lsamp3,acf1,acf2,acf3,acfamp1,acfamp2,acfamp3
int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
117935303,13,0.0018316209316253664,3.7441029788381184,5.525001989020606,2.274048301268932,0.0042324812546760855,0.003801049689865179,0.0017535487620507484,0.0694445,0.07500006000000001,0.07222228,0.01477058380131474,0.014756393144739647,0.014742200479031298
118010415,13,0.0033662557601930048,10.663761828896098,7.485380904901776,2.3192659171651977,0.007021002768735549,0.002659233810989161,0.0017524567399611294,0.07222228,0.09027785,0.09444452,0.02829308275403821,0.025799902993370708,0.02563864755250311
118053599,13,0.03137373924255371,6.932589597683937,5.128358980513067,3.3811500352995587,0.7790142854090246,0.0946020476001007,0.09104618836139353,0.0694445,6.80139433,6.805561,0.9190830186168646,0.6209071596330339,0.6209057867285954
118054887,13,0.02897542715072632,2.645035351250982,2.342284227790356,3.05346238615815,0.01685398984976197,0.002929709316659785,0.002264096668025945,0.00416667,0.07083339,0.07638895,0.07954484211484705,0.022339768615663842,0.021949506079630917
118055540,13,0.004783421754836925,5.7679329713342415,9.373365503045434,4.472163189077795,0.1032360778001358,0.024314551053414117,0.015072241112760674,0.07222228,0.0694445,0.09305563,0.18075212795276804,0.18066979749302026,0.17979913011820925
118060883,13,0.0039537250995635995,3.481849632730828,4.368115235526761,2.985945016258633,0.025679204506716676,0.002366434928258501,0.0018518686527940016,0.07638895,0.07083339,0.08055562000000001,0.04489353434264285,0.044877044691616304,0.044649316871818084
118074571,8,0.0030632138252257857,0.5884521235860678,11.242992734705444,0.5703371307899922,0.12275529831900853,0.0423746088512277,0.03753652611527136,0.00277778,0.0694445,0.58611158,0.2774313113963845,0.1799757858255744,0.145681008004694
118074571,9,0.003200823068618797,0.5876305125728919,0.6083339173397341,0.5682195922483617,0.09019131591630122,0.008728656200310617,0.00709676890686698,0.0694445,0.59166714,0.58750047,0.08956603758167632,0.08414176893993404,0.08412875767767251
118074571,10,0.0031239092350006104,3.132241423054154,0.5863002769241161,2.4027939967994465,0.012342140736587494,0.00674340845501243,0.005697960624838172,0.0694445,0.20555572,0.22777796,0.12506716519035424,0.09303926417014274,0.08889806518072946


In [8]:
# opening original big stats
og = ascii.read('data/OLDbigstats_by_sector.fits')
og

ID,NUMsectors,sector,rvar,ls1,ls2,ls3,lsamp1,lsamp2,lsamp3,version,HIP,TYC,UCAC,TWOMASS,SDSS,ALLWISE,GAIA,APASS,KIC,objType,typeSrc,ra,dec,POSflag,pmRA,e_pmRA,pmDEC,e_pmDEC,PMflag,plx,e_plx,PARflag,gallong,gallat,eclong,eclat,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,TWOMflag,prox,w1mag,e_w1mag,w2mag,e_w2mag,w3mag,e_w3mag,w4mag,e_w4mag,GAIAmag,e_GAIAmag,Tmag,e_Tmag,TESSflag,SPFlag,Teff,e_Teff,logg,e_logg,MH,e_MH,rad,e_rad,mass,e_mass,rho,e_rho,lumclass,lum,e_lum,d,e_d,ebv,e_ebv,numcont,contratio,disposition,duplicate_id,priority,eneg_EBV,epos_EBV,EBVflag,eneg_Mass,epos_Mass,eneg_Rad,epos_Rad,eneg_rho,epos_rho,eneg_logg,epos_logg,eneg_lum,epos_lum,eneg_dist,epos_dist,distflag,eneg_Teff,epos_Teff,TeffFlag,gaiabp,e_gaiabp,gaiarp,e_gaiarp,gaiaqflag,starchareFlag,VmagFlag,BmagFlag,splists,e_RA,e_Dec,RA_orig,Dec_orig,e_RA_orig,e_Dec_orig,raddflag,wdflag,objID
int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str12,str10,str16,str1,str19,str19,str8,int64,str4,str10,float64,float64,str10,float64,float64,float64,float64,str6,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,int64,float64,str5,str1,float64,float64,float64,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,str6,float64,float64,float64,float64,int64,str1,str8,str8,str15,float64,float64,float64,float64,float64,float64,int64,int64,int64
117935303,1,13,0.0018316209316253664,3.7441029788381184,5.525001989020606,2.274048301268932,0.0042324812546760855,0.003801049689865179,0.0017535487620507484,20190415,--,8755-01228-1,157-195298,18051266-5837057,N,J180512.67-583705.9,6647718671011095680,N/A,--,STAR,tmgaia2,271.302779383338,-58.6182469259589,tmgaia2,4.89615,0.0590119,-23.3937,0.0619381,gaia2,7.44041,0.0461462,gaia2,335.220989192817,-17.2435744099619,270.829994331233,-35.1827190969001,9.688,0.039,9.117,0.003,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,8.211,0.029,7.956,0.033,7.927,0.038,AAA-111-111-000-0-0,nan,7.864,0.027,7.884,0.021,7.854,0.018,8.004,0.215,9.04906,0.000282,8.6869,0.006,rered,gaia2,6321.0,134.697,4.16952,0.0861593,nan,nan,1.52309,0.0662111,1.25,0.194881,0.353778,0.0823205,DWARF,3.33649659,0.1421345,133.889,0.8335,0.0237361,0.01036807,627,0.008751403,N/A,N,0.00239241053,0.00833304,0.0124031,schlegel,0.176539,0.213224,0.0759362,0.0564859,0.0823694,0.0822717,0.0984868,0.0738318,0.153813,0.130456,0.828,0.839,bj2018,145.799,123.595,dered,9.33075,0.00121,8.64378,0.000882,1,N,tycho2v3,tycho2b3,N/A,1.04958812582618,0.96073247115476,271.302819865606,-58.618347648841,0.0362761076718781,0.0364557743962286,1,0,1420035826
118010415,1,13,0.0033662557601930048,10.663761828896098,7.485380904901776,2.3192659171651977,0.007021002768735549,0.002659233810989161,0.0017524567399611294,20190415,--,8751-02247-1,167-191417,18053362-5639566,N,J180533.60-563957.7,6651872243685919360,29458630,--,STAR,tmgaia2,271.390094426986,-56.6657534462633,tmgaia2,-16.3723,0.0605407,-96.8914,0.0523163,gaia2,7.15085,0.0378829,gaia2,337.100330070397,-16.4821426841295,270.913176207169,-33.230878549371,11.26,0.119,10.513,0.008,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,9.198,0.027,8.967,0.042,8.832,0.021,AAA-112-111-000-0-0,nan,8.765,0.022,8.807,0.02,8.731,0.025,8.337,nan,10.3267,0.000486,9.8536,0.006,rered,gaia2,5623.0,123.445,4.32845,0.0766169,nan,nan,1.13452,0.0550226,1.0,0.126666,0.684805,0.144612,DWARF,1.15928245,0.03864105,139.286,0.7405,0.0278073,0.01051959,690,0.164200917,N/A,N,0.00192848686,0.012734,

In [9]:
#comparing ids to make sure theyre the same

notin_b, notin_a = returnNotMatches(og['ID'], cvzstat_table['ID']) #a,b
len(notin_b),len(notin_a)#, notin_b

(166, 0)

In [10]:
#confirms (in 2ways) that these werent able to get an acf - not lk objects

# import glob
# for tic in noLK:
#     fullpath = glob.glob('/Volumes/Seagate-stars/LightCurves/*{}-*-s_lc.fits'.format(tic))
#     for file in fullpath:
#         try:
#             lcfile = lk.open(file)
#             print('HEY I WORKED!!',tic, file)
#         except ValueError as err:
#             print(err)

noLK = [141867895,141870589,141910606,141974066,141975926,141976247,149037112,149039372,149060586,149102762,149103941,149154927,149156139,149158724]
notin_b2, notin_a2 = returnNotMatches(notin_b, noLK) #a,b
len(notin_b2),len(notin_a2)

(0, 0)

In [11]:
#joining the tables
bigstats = join(og, cvzstat_table)
bigstats #note ACF values are LAST columns

ID,NUMsectors,sector,rvar,ls1,ls2,ls3,lsamp1,lsamp2,lsamp3,version,HIP,TYC,UCAC,TWOMASS,SDSS,ALLWISE,GAIA,APASS,KIC,objType,typeSrc,ra,dec,POSflag,pmRA,e_pmRA,pmDEC,e_pmDEC,PMflag,plx,e_plx,PARflag,gallong,gallat,eclong,eclat,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,TWOMflag,prox,w1mag,e_w1mag,w2mag,e_w2mag,w3mag,e_w3mag,w4mag,e_w4mag,GAIAmag,e_GAIAmag,Tmag,e_Tmag,TESSflag,SPFlag,Teff,e_Teff,logg,e_logg,MH,e_MH,rad,e_rad,mass,e_mass,rho,e_rho,lumclass,lum,e_lum,d,e_d,ebv,e_ebv,numcont,contratio,disposition,duplicate_id,priority,eneg_EBV,epos_EBV,EBVflag,eneg_Mass,epos_Mass,eneg_Rad,epos_Rad,eneg_rho,epos_rho,eneg_logg,epos_logg,eneg_lum,epos_lum,eneg_dist,epos_dist,distflag,eneg_Teff,epos_Teff,TeffFlag,gaiabp,e_gaiabp,gaiarp,e_gaiarp,gaiaqflag,starchareFlag,VmagFlag,BmagFlag,splists,e_RA,e_Dec,RA_orig,Dec_orig,e_RA_orig,e_Dec_orig,raddflag,wdflag,objID,acf1,acf2,acf3,acfamp1,acfamp2,acfamp3
int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str12,str10,str16,str1,str19,str19,str8,int64,str4,str10,float64,float64,str10,float64,float64,float64,float64,str6,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,int64,float64,str5,str1,float64,float64,float64,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,str6,float64,float64,float64,float64,int64,str1,str8,str8,str15,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64
24671921,1,13,0.0071094632148742684,4.714723379698388,6.56151084234894,3.710270948955808,0.3659571002372538,0.07985129054555644,0.04503873001476469,20190415,--,9147-00479-1,088-006118,02174757-7225033,N,J021747.55-722503.3,4644568429780642432,33528137,--,STAR,tmgaia2,34.4482217510531,-72.4176001947251,tmgaia2,-3.65382,0.0364772,5.77749,0.0357242,gaia2,2.97014,0.0214375,gaia2,294.160388804651,-43.2094271995752,318.2388411903,-70.4911761580997,11.906,0.175,11.455,0.012,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,10.328,0.026,10.029,0.025,9.986,0.025,AAA-222-111-000-0-0,nan,9.915,0.023,9.942,0.02,9.856,0.039,8.699,nan,11.276,0.000364,10.8858,0.006,rered,spect,6404.07,107.76,4.30389,0.0854392,-0.6,0.12,1.32552,0.0547688,1.29,0.201841,0.553901,0.122882,DWARF,2.66250777,0.1229225,333.453,2.402,0.0242846,0.0100505846,77,0.00598344952,N/A,N,0.00145012152,0.0103777,0.00972347,schlegel,0.138792,0.26489,0.0556266,0.0539111,0.110996,0.134767,0.0703183,0.10056,0.122903,0.122942,2.385,2.419,bj2018,128.371,87.1495,spect,11.5725,0.001416,10.8315,0.001285,1,N,tycho2v3,tycho2b3,N/A,0.701064465211099,0.554017317754182,34.4481696725391,-72.4175753194275,0.0217012028773991,0.0179917203604472,1,0,518150217,0.0694445,0.10138897,0.11250009,0.4609423773575478,0.4577526606916515,0.4567945412636466
24672008,3,1,0.012634336948394777,0.7286541899419359,2.4993476637364016,0.8424150799198755,0.0006754832200712138,0.0006170244614573783,0.0005852819530078737,20190415,--,N/A,088-006116,02174403-7234346,N,J021744.49-723432.9,4644553478998527104,33528124,--,STAR,tmgaia2,34.4336744945758,-72.5762402095853,tmgaia2,178.354,0.0331487,131.544,0.0284528,gaia2,11.6551,0.0169377,gaia2,294.263282412239,-43.0696104538473,317.769209310265,-70.5160558879286,15.634,0.024,14.134,0.04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,11.025,0.026,10.474,0.029,10.246,0.025,AAA-222-111-000-0-0,nan,10.065,0.023,9.999,0.02,9.809,0.035,8.892,nan,13.3904,0.000448,12.3603,0.00735645,cdwrf,cdwrf,3735.0,157.0,4.72886,0.0086

# Uncomment here to remake table

In [12]:
#saving sector table
# ascii.write(bigstats,'data/bigstats_by_sector.fits',overwrite=True) #already ran


In [13]:
shortstats = bigstats[bigstats['NUMsectors']>=10]
len(shortstats)

70865

In [14]:
seen, uniqs = uniquefinder(shortstats['ID'])

there are 5995 unique tics. Use the first output dictionary to see number of occurances for duplicates


In [15]:
#verifying lk works---passed

# tics=[118074571,24694131,141867895] #expect YES, YES, no
# import glob
# for tic in tics:
#     fullpath = glob.glob('/Volumes/Seagate-stars/LightCurves/*{}-*-s_lc.fits'.format(tic))
#     for file in fullpath:
#         try:
#             lcfile = lk.open(file)
#             print('HEY I WORKED!!',tic, file)
#         except ValueError as err:
#             print(err)



# Combining bigstats with BLS stats for stitched lcs

In [2]:
#load data
bigstats = ascii.read('data/bigstats_stitchedlcs.fits')
bls_stats = pd.read_csv('data/bls_stats.csv')
# master_stats = ascii.read('data/master_bls_bigstats_stitchedlcs.fits')



In [15]:
# match catalogs

bigstats.columns,list(bls_stats.columns)

#convert df to table
bls_table = Table.from_pandas(bls_stats)
#rename colname to match
bls_table.rename_column('tic','ID')
#match based on tic id 
master_table = join(bigstats, bls_table, keys='ID')
print('check these lengths still:',len(master_table),len(bls_stats),len(bigstats))

check these lengths: 24071 24069 24087


In [16]:
#save new master table

master_table
#ascii.write(master_table,'data/master_bls_bigstats_stitchedlcs.fits',overwrite=True) #already ran 

# TESTS

In [16]:
seen, uniqs = uniquefinder(cvzstat_table['ID'])
len(tic_list)

there are 24071 unique tics. Use the first output dictionary to see number of occurances for duplicates


24085

In [17]:
#uses all dled tics
notin_b, notin_a = returnNotMatches(tic_list, cvzstat_table['ID']) #a,b

print('figured out these tics werent files lk could open')
len(notin_b),len(notin_a), notin_b

figured out these tics werent files lk could open


(14,
 0,
 [141867895,
  141870589,
  141910606,
  141974066,
  141975926,
  141976247,
  149037112,
  149039372,
  149060586,
  149102762,
  149103941,
  149154927,
  149156139,
  149158724])

# ~~PYTHONPATH problems~~ Solved!

In [18]:
#import starspot as ss

In [19]:
#test ss module -- passed

# time = [1,2,3,4,5,6,7,8,9,10]
# flux = [1,2,3,2.5,1.1,2,3.1,1.9,1,2]
# unc = [1,1,1,1,1,1,1,1,1,1]

# model = ss.RotationModel(time,flux,unc)
# rp = model.ls_rotation()
# print(rp)

In [20]:
#to find paths
# import sys
# for p in sys.path:
#     print(p)
# print(sys.path) #may not contain all paths & above method is easier to read

In [21]:
#if i had needed to add any paths
# sys.path.append('/Users/helenfellow/opt/anaconda3/lib/python3.7/site-packages/starspot-0.1-py3.7.egg
# ')

In [22]:
# to get environoment
# import os
# os.environ['PYTHONPATH'].split(os.pathsep) #error b/c this isnt set yet